<a href="https://colab.research.google.com/github/rajesh-bhat/data-aisummit-2021-databricks-conversational-ai/blob/main/DistilBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Thu May 27 14:52:19 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip3 install tqdm
!pip3 install transformers

     |████████████████████████████████| 2.3MB 8.5MB/s 
     |████████████████████████████████| 3.3MB 49.2MB/s 
     |████████████████████████████████| 901kB 47.1MB/s 


In [3]:
!gdown --id 1OlcvGWReJMuyYQuOZm149vHWwPtlboR6 --output train.csv
!gdown --id 1Oi5cRlTybuIF2Fl5Bfsr-KkqrXrdt77w --output valid.csv
!gdown --id 1ep9H6-HvhB4utJRLVcLzieWNUSG3P_uF --output test.csv

Downloading...
From: https://drive.google.com/uc?id=1OlcvGWReJMuyYQuOZm149vHWwPtlboR6
To: /content/train.csv
100% 799k/799k [00:00<00:00, 3.75MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Oi5cRlTybuIF2Fl5Bfsr-KkqrXrdt77w
To: /content/valid.csv
100% 43.3k/43.3k [00:00<00:00, 1.40MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ep9H6-HvhB4utJRLVcLzieWNUSG3P_uF
To: /content/test.csv
100% 43.1k/43.1k [00:00<00:00, 1.39MB/s]


In [41]:
import numpy as np
import pandas as pd
import tensorflow as tf
import transformers
import matplotlib.pyplot as plt
import tqdm
from sklearn.preprocessing import LabelBinarizer

%matplotlib inline

# fix random seed for reproducibility
seed = 222
np.random.seed(seed)
tf.random.set_seed(seed)

print("TF Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

TF Version:  2.5.0
Eager mode:  True
GPU is available


##Overview of data

In [42]:
train = pd.concat([pd.read_csv(file) for file in ["train.csv","valid.csv"]])
train = train.groupby('intent').sample(frac=0.25)

test = pd.read_csv("test.csv")

print(train.shape)
print(test.shape)

train.head()

(3447, 2)
(700, 2)


,text,intent
1506,add the rebirth of kirk franklin to the fiesta...,AddToPlaylist
1587,add d flame to the dcode2016 playlist,AddToPlaylist
6634,put four in blue onto ultimate 00s,AddToPlaylist
825,add vyechnyy strannik to lo que suena new york,AddToPlaylist
6446,put this whole album on the playlist called ol...,AddToPlaylist


In [43]:
test.head()

,text,intent
0,add sabrina salerno to the grime instrumentals...,AddToPlaylist
1,i want to bring four people to a place that s ...,BookRestaurant
2,put lindsey cardinale into my hillary clinton ...,AddToPlaylist
3,will it snow in mt on june 13 2038,GetWeather
4,play signe anderson chant music that is newest,PlayMusic


In [44]:
train.intent.value_counts()

PlayMusic               504
GetWeather              499
BookRestaurant          495
RateBook                494
SearchScreeningEvent    488
SearchCreativeWork      487
AddToPlaylist           480
Name: intent, dtype: int64

In [45]:
intent_mapping = {x:idx for idx,x in enumerate(train.intent.unique().tolist())}
train['target'] = train['intent'].map(intent_mapping)
test['target'] = test['intent'].map(intent_mapping)
train.head()

,text,intent,target
1506,add the rebirth of kirk franklin to the fiesta...,AddToPlaylist,0
1587,add d flame to the dcode2016 playlist,AddToPlaylist,0
6634,put four in blue onto ultimate 00s,AddToPlaylist,0
825,add vyechnyy strannik to lo que suena new york,AddToPlaylist,0
6446,put this whole album on the playlist called ol...,AddToPlaylist,0


In [46]:
test.head()    

,text,intent,target
0,add sabrina salerno to the grime instrumentals...,AddToPlaylist,0
1,i want to bring four people to a place that s ...,BookRestaurant,1
2,put lindsey cardinale into my hillary clinton ...,AddToPlaylist,0
3,will it snow in mt on june 13 2038,GetWeather,2
4,play signe anderson chant music that is newest,PlayMusic,3


In [51]:
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [52]:
def create_bert_input_features(tokenizer, docs, max_seq_length):

    all_ids, all_masks = [], []

    for doc in tqdm.tqdm(docs, desc="Converting docs to features"):
        
        tokens = tokenizer.tokenize(doc)

        if len(tokens) > max_seq_length-2:
            tokens = tokens[0 : (max_seq_length-2)]
        
        tokens = ['[CLS]'] + tokens + ['[SEP]']
        ids = tokenizer.convert_tokens_to_ids(tokens)
        masks = [1] * len(ids)

        # Zero-pad up to the sequence length.
        while len(ids) < max_seq_length:
            ids.append(0)
            masks.append(0)
        
        all_ids.append(ids)
        all_masks.append(masks)
    
    encoded = np.array([all_ids, all_masks])
    
    return encoded

In [53]:
MAX_SEQ_LENGTH = 256
num_labels = 7

inp_id = tf.keras.layers.Input(shape=(MAX_SEQ_LENGTH,), dtype='int32', name="bert_input_ids")
inp_mask = tf.keras.layers.Input(shape=(MAX_SEQ_LENGTH,), dtype='int32', name="bert_input_masks")
inputs = [inp_id, inp_mask]

hidden_state = transformers.TFDistilBertModel.from_pretrained('distilbert-base-uncased')(inputs)[0]

pooled_output = hidden_state[:, 0]

dense1 = tf.keras.layers.Dense(256, activation='relu')(pooled_output)
drop1 = tf.keras.layers.Dropout(0.25)(dense1)
dense2 = tf.keras.layers.Dense(256, activation='relu')(drop1)
drop2 = tf.keras.layers.Dropout(0.25)(dense2)
output = tf.keras.layers.Dense(num_labels, activation='sigmoid')(drop2)

model = tf.keras.Model(inputs=inputs, outputs=output)
model.compile(optimizer=tf.optimizers.Adam(learning_rate=2e-5, 
                                           epsilon=1e-08), 
              loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_layer_norm', 'vocab_projector', 'activation_13', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
bert_input_ids (InputLayer)     [(None, 256)]        0                                            
__________________________________________________________________________________________________
bert_input_masks (InputLayer)   [(None, 256)]        0                                            
__________________________________________________________________________________________________
tf_distil_bert_model_1 (TFDisti TFBaseModelOutput(la 66362880    bert_input_ids[0][0]             
                                                                 bert_input_masks[0][0]           
__________________________________________________________________________________________________
tf.__operators__.getitem_1 (Sli (None, 768)          0           tf_distil_bert_model_1[0][0

In [54]:
train_features_ids, train_features_masks = create_bert_input_features(tokenizer, train["text"], max_seq_length=MAX_SEQ_LENGTH)
val_features_ids, val_features_masks = create_bert_input_features(tokenizer, test["text"], max_seq_length=MAX_SEQ_LENGTH)

print('Train Features:', train_features_ids.shape, train_features_masks.shape)
print('Val Features:', val_features_ids.shape, val_features_masks.shape)

Converting docs to features: 100%|██████████| 700/700 [00:00<00:00, 3281.16it/s]

Train Features: (3447, 256) (3447, 256)
Val Features: (700, 256) (700, 256)


In [55]:
enc = LabelBinarizer( )

In [56]:
enc.fit(train["target"])
enc.classes_

y_train = enc.transform(train["target"])
y_test = enc.transform(test["target"])

In [38]:
y_train.shape, y_test.shape

((3447, 7), (700, 7))

In [57]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                      patience=3,
                                      restore_best_weights=True,
                                      verbose=1)

model.fit([train_features_ids, 
           train_features_masks], y_train, 
          validation_data=([val_features_ids, 
                            val_features_masks], y_test),
          epochs=3, 
          batch_size=32, 
          shuffle=True,
          callbacks=[es],
          verbose=1)

Epoch 1/3
108/108 [==============================] - 129s 884ms/step - loss: 0.7201 - accuracy: 0.7778 - val_loss: 0.1515 - val_accuracy: 0.9514
Epoch 2/3
108/108 [==============================] - 94s 869ms/step - loss: 0.0884 - accuracy: 0.9785 - val_loss: 0.1074 - val_accuracy: 0.9686
Epoch 3/3
108/108 [==============================] - 94s 868ms/step - loss: 0.0377 - accuracy: 0.9922 - val_loss: 0.1370 - val_accuracy: 0.9700


In [ ]:
model.save_weights('distillbert_ft_wts.h5')

In [ ]:
test_features_ids, test_features_masks = create_bert_input_features(tokenizer, test_reviews, 
                                                                    max_seq_length=MAX_SEQ_LENGTH)

print('Test Features:', test_features_ids.shape, test_features_masks.shape)

Converting docs to features: 100%|██████████| 40000/40000 [03:16<00:00, 203.42it/s]


Test Features: (40000, 500) (40000, 500)


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

predictions = [1 if pr > 0.5 else 0 
                   for pr in model.predict([test_features_ids, 
                                            test_features_masks], batch_size=200, verbose=0).ravel()]

print("Accuracy: %.2f%%" % (accuracy_score(test_sentiments, predictions)*100))
print(classification_report(test_sentiments, predictions))
pd.DataFrame(confusion_matrix(test_sentiments, predictions))

Accuracy: 90.71%
             precision    recall  f1-score   support

          0       0.90      0.91      0.91     20028
          1       0.91      0.90      0.91     19972

avg / total       0.91      0.91      0.91     40000



,0,1
0,18248,1780
1,1937,18035
